In [3]:
import tensorflow as tf
import pandas as pd

path = "./autoencoder/single_axis"
accel_x_model = tf.saved_model.load(f'{path}/accel_x')
accel_y_model = tf.saved_model.load(f'{path}/accel_y')
accel_z_model = tf.saved_model.load(f'{path}/accel_z')
accel_x_simple_model = tf.saved_model.load(f'{path}/accel_x_simple')
accel_y_simple_model = tf.saved_model.load(f'{path}/accel_y_simple')
accel_z_simple_model = tf.saved_model.load(f'{path}/accel_z_simple')
gyro_x_model = tf.saved_model.load(f'{path}/gyro_x')
gyro_y_model = tf.saved_model.load(f'{path}/gyro_y')
gyro_z_model = tf.saved_model.load(f'{path}/gyro_z')

file = "../data/Raw/logRaws/20230916_01_16_54_377_1018_logRaws.csv"
df_input = pd.read_csv(file, usecols=[16, 17, 18, 22, 23, 24], header=None)
df_input.columns = ['accel_x', 'accel_y', 'accel_z',
              'gyro_x', 'gyro_y', 'gyro_z']
df_input.astype('float32')

df_output = pd.read_csv(file, usecols=[19, 20, 21, 25, 26, 27], header=None)
df_output.columns = ['accel_x_given', 'accel_y_given', 'accel_z_given',
                     'gyro_x_given', 'gyro_y_given', 'gyro_z_given']
df_output.astype('float32')

df_input_norm = df_input.copy()
df_input_norm /= [16, 16, 16, 2000, 2000, 2000]

data_accel_x = tf.constant(df_input_norm['accel_x'].values.reshape(1, 1000, 1), dtype='float32')
data_accel_y = tf.constant(df_input_norm['accel_y'].values.reshape(1, 1000, 1), dtype='float32')
data_accel_z = tf.constant(df_input_norm['accel_z'].values.reshape(1, 1000, 1), dtype='float32')
data_gyro_x = tf.constant(df_input_norm['gyro_x'].values.reshape(1, 1000, 1), dtype='float32')
data_gyro_y = tf.constant(df_input_norm['gyro_y'].values.reshape(1, 1000, 1), dtype='float32')
data_gyro_z = tf.constant(df_input_norm['gyro_z'].values.reshape(1, 1000, 1), dtype='float32')

output_accel_x = accel_x_model(data_accel_x)
output_accel_y = accel_y_model(data_accel_y)
output_accel_z = accel_z_model(data_accel_z)
output_accel_x_simple = accel_x_simple_model(data_accel_x)
output_accel_y_simple = accel_y_simple_model(data_accel_y)
output_accel_z_simple = accel_z_simple_model(data_accel_z)
output_gyro_x = gyro_x_model(data_gyro_x)
output_gyro_y = gyro_y_model(data_gyro_y)
output_gyro_z = gyro_z_model(data_gyro_z)

# create a dataframe with the output
columns = ['frame', 'accel_x_input', 'accel_x_given', 'accel_x_output', 'accel_x_simple_output',
              'accel_y_input', 'accel_y_given', 'accel_y_output', 'accel_y_simple_output',
              'accel_z_input', 'accel_z_given', 'accel_z_output', 'accel_z_simple_output',
              'gyro_x_input', 'gyro_x_given', 'gyro_x_output',
              'gyro_y_input', 'gyro_y_given', 'gyro_y_output',
              'gyro_z_input', 'gyro_z_given', 'gyro_z_output']

df = pd.DataFrame(columns=columns)
df['frame'] = df_input.index
df['accel_x_input'] = df_input['accel_x']
df['accel_x_given'] = df_output['accel_x_given']
df['accel_x_output'] = output_accel_x[0, :, 0] * 16
df['accel_x_simple_output'] = output_accel_x_simple[0, :, 0] * 16
df['accel_y_input'] = df_input['accel_y']
df['accel_y_given'] = df_output['accel_y_given']
df['accel_y_output'] = output_accel_y[0, :, 0] * 16
df['accel_y_simple_output'] = output_accel_y_simple[0, :, 0] * 16
df['accel_z_input'] = df_input['accel_z']
df['accel_z_given'] = df_output['accel_z_given']
df['accel_z_output'] = output_accel_z[0, :, 0] * 16
df['accel_z_simple_output'] = output_accel_z_simple[0, :, 0] * 16
df['gyro_x_input'] = df_input['gyro_x']
df['gyro_x_given'] = df_output['gyro_x_given']
df['gyro_x_output'] = output_gyro_x[0, :, 0] * 2000
df['gyro_y_input'] = df_input['gyro_y']
df['gyro_y_given'] = df_output['gyro_y_given']
df['gyro_y_output'] = output_gyro_y[0, :, 0] * 2000
df['gyro_z_input'] = df_input['gyro_z']
df['gyro_z_given'] = df_output['gyro_z_given']
df['gyro_z_output'] = output_gyro_z[0, :, 0] * 2000

df.to_csv('./sample_output.csv', index=False)
